# Author: Tanay Yadav
# Decision Tree
# AI2000 - Foundations of Machine Learning

In [8]:
# Normal Implementation

# importing required directories
import numpy as np
import csv


my_name = "Tanay-Madhav-Yadav"


# using entropy to select the best split

def entropy(e_data):
    wine_qual = len(e_data[0]) - 1            # index for the quality of wine (1/0)
    num_0 = 0
    num_1 = 0
    for i in e_data:                          # calculating the instances of '1'
        if i[wine_qual] == 1:
            num_1 += 1
        else:
            num_0 += 1
    p0 = num_0 / (num_0 + num_1)
    p1 = 1 - p0
    if p0 != 0:
        p0 = -p0 * np.log2(p0)                 # calculation of entropy
    if p1 != 0:
        p1 = -p1 * np.log2(p1)
    return p0 + p1


# function to find the best possible split according to features and thresholds.

def best_split(s_data):
    best_split_dict = {'feature': None, 'threshold': None, 'left': [], 'right': []}    # dictionary to store the info
    info_gain = -float('inf') + 1                                                      # about the best split
    r_len = len(s_data[0]) - 1                # not considering the 'wine quality' column as feature
    max_info_gain = -float('inf')

    for feature in range(r_len):
        values = [s_data[x][feature] for x in range(len(s_data))]
        max_val = max(values)
        min_val = min(values)
        p_thresholds = np.linspace(min_val, max_val, 10)                     # creating abstract thresholds

        for threshold in p_thresholds:
            data_left = []                                                   # list to store the split data
            data_right = []
            for inputs in s_data:
                if inputs[feature] < threshold:
                    # dropping the feature utilised to split
                    data_left.append([inputs[x] for x in [*range(0, feature), *range(feature + 1, r_len + 1)]])
                else:
                    data_right.append([inputs[x] for x in [*range(0, feature), *range(feature + 1, r_len + 1)]])

            if len(data_left) > 0 and len(data_right) > 0:
                # finding the best split by calculating information gain on each possible split.
                info_gain = entropy(s_data) - ((len(data_left) / len(s_data)) * entropy(data_left) + (
                        len(data_right) / len(s_data)) * entropy(data_right))
            if info_gain > max_info_gain:
                # storing the parameters for split into the dictionary
                max_info_gain = info_gain
                best_split_dict['feature'] = feature
                best_split_dict['threshold'] = threshold
                best_split_dict['left'] = data_left
                best_split_dict['right'] = data_right
    return best_split_dict


# Implement your decision tree below
class DecisionTree:
    tree = {}

    def learn(self, t_data):
        # creating a tree using dictionary
        tree = {'feature': None, 'value': None, 'threshold': None, 'left_tree': {}, 'right_tree': {}}
        if len(t_data) > 0:
            # checking if at least 2 features are available to differentiate and split and entropy is non-zero.
            if len(t_data[0]) >= 2 and entropy(t_data) != 0:
                b_split = best_split(t_data)
                tree['feature'] = b_split['feature']
                tree['threshold'] = b_split['threshold']

                # recursively building the left and right sub-trees
                left_tree = DecisionTree.learn(self, b_split['left'])
                right_tree = DecisionTree.learn(self, b_split['right'])
                tree['right_tree'] = right_tree
                tree['left_tree'] = left_tree
                return tree

            # if there is just 1 feature remaining, it is a leaf
            else:
                tree['left_tree'] = None
                tree['right_tree'] = None
                label_idx = len(t_data[0]) - 1
                label = [t_data[x][label_idx] for x in range(len(t_data))]
                tree['value'] = max(label, key=label.count)
                return tree

    def classify(self, test_data, tree):
        # ensuring that the tree has been trained
        if tree:

            # checking whether we have reached the leaf node
            if tree['value'] is not None:
                return tree['value']
            feature_val = test_data[tree['feature']]

            # classifying based on thresholds set during best_split()
            if feature_val < tree['threshold']:
                return self.classify(
                    # leaving out already classified feature to increase accuracy.
                    [test_data[x] for x in [*range(0, tree['feature']), *range(tree['feature'] + 1, len(test_data))]],
                    tree['left_tree'])
            else:
                return self.classify(
                    [test_data[x] for x in [*range(0, tree['feature']), *range(tree['feature'] + 1, len(test_data))]],
                    tree['right_tree'])


def run_decision_tree():
    average_accu = 0
    # Load data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = []
        for line in csv.reader(f, delimiter=","):
            row = [float(x) for x in line]
            data.append(row)
    print("Number of records: %d" % len(data))

    # Split training/test sets
    K = 10
    for k in range(0, K):
        training_set = [x for i, x in enumerate(data) if i % K != k]
        test_set = [x for i, x in enumerate(data) if i % K == k]

        tree = DecisionTree()
        # Construct a tree using training set
        Tree = tree.learn(training_set)

        # Classify the test set using the tree we just constructed
        results = []
        for instance in test_set:
            result = tree.classify(instance[:-1], Tree)
            results.append(result == instance[-1])

        # Accuracy
        accuracy = float(results.count(True)) / float(len(results))
        average_accu += accuracy
        print('Fold:', k + 1, "/ 10, accuracy: %.4f" % accuracy)

        # Writing results to a file (DO NOT CHANGE)
        f = open(my_name + "result.txt", "w")
        f.write("accuracy: %.4f" % accuracy)
        f.close()

    print('For Entropy based implementation, Average Accuracy = %.4f' % (average_accu / K))


if __name__ == "__main__":
    run_decision_tree()

Number of records: 4898
Fold: 1 / 10, accuracy: 0.8184
Fold: 2 / 10, accuracy: 0.8408
Fold: 3 / 10, accuracy: 0.7816
Fold: 4 / 10, accuracy: 0.8265
Fold: 5 / 10, accuracy: 0.8367
Fold: 6 / 10, accuracy: 0.7939
Fold: 7 / 10, accuracy: 0.8224
Fold: 8 / 10, accuracy: 0.8102
Fold: 9 / 10, accuracy: 0.7648
Fold: 10 / 10, accuracy: 0.8282
For Entropy based implementation, Average Accuracy = 0.8124


In [19]:
# Gini based classification

# importing required directories
import numpy as np
import csv


my_name = "Tanay-Madhav-Yadav"


# using gini to select the best split

def gini(e_data):
    wine_qual = len(e_data[0]) - 1            # index for the quality of wine (1/0)
    num_0 = 0
    num_1 = 0
    for i in e_data:                          # calculating the instances of '1'
        if i[wine_qual] == 1:
            num_1 += 1
        else:
            num_0 += 1
    p0 = num_0 / (num_0 + num_1)
    p1 = 1 - p0
    if p0 != 0:
        p0 = p0 ** 2                 # calculation of gini index
    if p1 != 0:
        p1 = p1 ** 2
    return 1 - p0 - p1


# function to find the best possible split according to features and thresholds.

def best_split(s_data):
    best_split_dict = {'feature': None, 'threshold': None, 'left': [], 'right': []}    # dictionary to store the info
    info_gain = -float('inf') + 1                                                      # about the best split
    r_len = len(s_data[0]) - 1                # not considering the 'wine quality' column as feature
    max_info_gain = -float('inf')

    for feature in range(r_len):
        values = [s_data[x][feature] for x in range(len(s_data))]
        max_val = max(values)
        min_val = min(values)
        p_thresholds = np.linspace(min_val, max_val, 10)                     # creating abstract thresholds

        for threshold in p_thresholds:
            data_left = []                                                   # list to store the split data
            data_right = []
            for inputs in s_data:
                if inputs[feature] < threshold:
                    # dropping the feature utilised to split
                    data_left.append([inputs[x] for x in [*range(0, feature), *range(feature + 1, r_len + 1)]])
                else:
                    data_right.append([inputs[x] for x in [*range(0, feature), *range(feature + 1, r_len + 1)]])

            if len(data_left) > 0 and len(data_right) > 0:
                # finding the best split by calculating information gain on each possible split.
                info_gain = gini(s_data) - ((len(data_left) / len(s_data)) * gini(data_left) + (
                        len(data_right) / len(s_data)) * gini(data_right))
            if info_gain > max_info_gain:
                # storing the parameters for split into the dictionary
                max_info_gain = info_gain
                best_split_dict['feature'] = feature
                best_split_dict['threshold'] = threshold
                best_split_dict['left'] = data_left
                best_split_dict['right'] = data_right
    return best_split_dict


# Implement your decision tree below
class DecisionTree:
    tree = {}

    def learn(self, t_data):
        # creating a tree using dictionary
        tree = {'feature': None, 'value': None, 'threshold': None, 'left_tree': {}, 'right_tree': {}}
        if len(t_data) > 0:
            # checking if at least 2 features are available to differentiate and split and gini enables split.
            if len(t_data[0]) >= 2 and gini(t_data) > 0 and gini(t_data) < 0.5:
                b_split = best_split(t_data)
                tree['feature'] = b_split['feature']
                tree['threshold'] = b_split['threshold']

                # recursively building the left and right sub-trees
                left_tree = DecisionTree.learn(self, b_split['left'])
                right_tree = DecisionTree.learn(self, b_split['right'])
                tree['right_tree'] = right_tree
                tree['left_tree'] = left_tree
                return tree

            # if there is just 1 feature remaining, it is a leaf
            else:
                tree['left_tree'] = None
                tree['right_tree'] = None
                label_idx = len(t_data[0]) - 1
                label = [t_data[x][label_idx] for x in range(len(t_data))]
                tree['value'] = max(label, key=label.count)
                return tree

    def classify(self, test_data, tree):
        # ensuring that the tree has been trained
        if tree:

            # checking whether we have reached the leaf node
            if tree['value'] is not None:
                return tree['value']
            feature_val = test_data[tree['feature']]

            # classifying based on thresholds set during best_split()
            if feature_val < tree['threshold']:
                return self.classify(
                    # leaving out already classified feature to increase accuracy.
                    [test_data[x] for x in [*range(0, tree['feature']), *range(tree['feature'] + 1, len(test_data))]],
                    tree['left_tree'])
            else:
                return self.classify(
                    [test_data[x] for x in [*range(0, tree['feature']), *range(tree['feature'] + 1, len(test_data))]],
                    tree['right_tree'])


def run_decision_tree():
    average_accu = 0
    # Load data set
    with open("wine-dataset.csv") as f:
        next(f, None)
        data = []
        for line in csv.reader(f, delimiter=","):
            row = [float(x) for x in line]
            data.append(row)
    print("Number of records: %d" % len(data))

    # Split training/test sets
    K = 10
    for k in range(0, K):
        training_set = [x for i, x in enumerate(data) if i % K != k]
        test_set = [x for i, x in enumerate(data) if i % K == k]

        tree = DecisionTree()
        # Construct a tree using training set
        Tree = tree.learn(training_set)

        # Classify the test set using the tree we just constructed
        results = []
        for instance in test_set:
            result = tree.classify(instance[:-1], Tree)
            results.append(result == instance[-1])

        # Accuracy
        accuracy = float(results.count(True)) / float(len(results))
        average_accu += accuracy
        print('Fold:', k + 1, "/ 10, accuracy: %.4f" % accuracy)

        # Writing results to a file (DO NOT CHANGE)
        f = open(my_name + "result.txt", "w")
        f.write("accuracy: %.4f" % accuracy)
        f.close()

    print('For Gini-Index based implementation, Average Accuracy = %.4f' % (average_accu / K))


if __name__ == "__main__":
    run_decision_tree()

Number of records: 4898
Fold: 1 / 10, accuracy: 0.8082
Fold: 2 / 10, accuracy: 0.8306
Fold: 3 / 10, accuracy: 0.7837
Fold: 4 / 10, accuracy: 0.8184
Fold: 5 / 10, accuracy: 0.8245
Fold: 6 / 10, accuracy: 0.8000
Fold: 7 / 10, accuracy: 0.8000
Fold: 8 / 10, accuracy: 0.8020
Fold: 9 / 10, accuracy: 0.7751
Fold: 10 / 10, accuracy: 0.8303
For Gini-Index based implementation, Average Accuracy = 0.8073


Base Accuracy: 81.24
Gini-Index Accuracy: 80.73

The accuracy over Gini has decreased due to a few outliers, but gini-index is consistently accurate over the 10-folds of the cross validation performed in this code. This is because the Gini-index is specialised to handle univariate splits and information gain just calculates the entropy of the data before the split compared to the data after the split. 